In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA
)
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT_REACT,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"}
)

In [2]:
a = "OK"

a += "bro" if True else "bruh"
a

'OKbro'

In [3]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflection_strategy=None,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

<PROMPT AGENT=====================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

---

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

---

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

---

Q: What profess

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<OUT AGENT=====================================================>
"Let's think step by step. Since VIVA Media AG changed its name in 2004, we need to find out what their new acronym stands for. After researching, it is found that their new acronym stands for VIVA Entertainment Group. So the answer is: VIVA Entertainment Group."
<OUT AGENT=====================================================>


TypeError: ReflexionCoTQAStrategy.create_output_dict() got an unexpected keyword argument 'key'

In [ ]:
out